In [87]:
import tweepy
import os
import json
import pandas as pd

In [21]:
with open('tweet_auth.json') as json_file:
    auth_creds = json.load(json_file)


# Variables that contains the credentials to access Twitter API
CONSUMER_KEY = auth_creds['consumer_key']
CONSUMER_SECRET = auth_creds['consumer_secret']
ACCESS_TOKEN = auth_creds['access_token']
ACCESS_SECRET = auth_creds['access_token_secret']


# Setup access to API
def connect_to_twitter_OAuth():
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    api = tweepy.API(auth)
    return api


# Create API object
api = connect_to_twitter_OAuth()

In [78]:
# Test credentials to pull my latest tweets and save to csv
    
latest_tweets = api.user_timeline(screen_name = 'stevendumbe', count=5, tweet_mode="extended")
data = []
for tweet in latest_tweets:
    try:
        data.append({"user_name": tweet._json['user']['screen_name'], "text": tweet._json['retweeted_status']['full_text']})
    except KeyError:  # Not a Retweet
        data.append({"user_name": tweet._json['user']['screen_name'], "text": tweet._json['full_text']})
print(data)
df = pd.json_normalize(data)
print(df)
df.to_csv('my_tweets.csv',index=False)

[{'user_name': 'stevendumbe', 'text': 'Bringing back the memories when @patrick_brandl and @stevendumbe met @RamyAshour . Thank you Ramy #Yalla https://t.co/WfUYPmYgmq'}, {'user_name': 'stevendumbe', 'text': 'Congratulations to @imperialcollege  PhD student and my friend @stevendumbe for winning the #WorldCup with @EnglandDodge in NYC #ItsComingHome https://t.co/OvD6D23tA8'}, {'user_name': 'stevendumbe', 'text': "Good luck to my friend @stevendumbe representing @EnglandDodge in the World Dodgeball Championship in #NYC. A PhD at @imperialcollege isn't enough. #ItsComingHome Check out @BBCSport https://t.co/Zyf0uk5obq and @EveningStandard https://t.co/FLidZ4agH6"}, {'user_name': 'stevendumbe', 'text': "We met @RamyAshour and we're never going to fail #TheArtist @BOsquash @joeybarrington #ICSquashOnTour https://t.co/205XAUqDtH"}, {'user_name': 'stevendumbe', 'text': "@PSAWorldTour just started watching and heard you guys putting Kidderminster down, Kidderminster is where it's at!"}]
     

In [94]:
class StreamListener(tweepy.StreamListener):
    def on_status(self, status):
        payload={}
        payload["username"]=status.user.screen_name
        payload["timestamp"]=str(status.created_at)
        if hasattr(status, "retweeted_status"):  # Check if Retweet
            try:
                payload["text"]=status.retweeted_status.extended_tweet["full_text"]
            except AttributeError:
                payload["text"]=status.retweeted_status.text
        else:
            try:
                payload["text"]=status.extended_tweet["full_text"]
            except AttributeError:
                payload["text"]=status.text
        print(payload)
#         print(json.dumps(payload))
    
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False

In [95]:
streamListener = StreamListener()
stream = tweepy.Stream(auth=api.auth, listener=streamListener, tweet_mode='extended')

tags = ["#london"]
stream.filter(track=tags)

{'username': 'southcartbooks1', 'timestamp': '2020-11-27 22:10:34', 'text': 'Newly listed today , hit our link to get this &amp; more \n\n#books #booksforsale #bookshop #booksale #walsall #westmidlands #Wolverhampton #birmingham #blackcountry #bookstagram #london #england #book #blackfriday #BlackFridayDeals #blackfridaysale \n\nhttps://t.co/gLUMA3LrfL https://t.co/MgJtRuEcOD'}
{"username": "southcartbooks1", "timestamp": "2020-11-27 22:10:34", "text": "Newly listed today , hit our link to get this &amp; more \n\n#books #booksforsale #bookshop #booksale #walsall #westmidlands #Wolverhampton #birmingham #blackcountry #bookstagram #london #england #book #blackfriday #BlackFridayDeals #blackfridaysale \n\nhttps://t.co/gLUMA3LrfL https://t.co/MgJtRuEcOD"}
{'username': 'FootyZoneNet', 'timestamp': '2020-11-27 22:10:40', 'text': '**WE ARE RECRUITING**\n\nPLAYERS &amp; COACHES WANTED FOR THE 2021-2022\n\n@echoleague @EssexCountyFA @EssexCorinthian #squadbooster #playerswanted #coacheswanted #

KeyboardInterrupt: 